In [15]:
import matplotlib as mpl
mpl.use('TkAgg')
import numpy as np
import pandas as pd
import torch as th
from torch.autograd import Variable as V
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from preprocessing import Preprocess_GAN,CreateBatch_GAN
# from model.model_class import Blackbox_IDS,Generator,Discriminator
import matplotlib.pyplot as plt

# https://github.com/HongQuangDevVN/IDSGAN-on-SDN/blob/master/model/save/BlackBox/IDS.pth

In [16]:
train_dataset = pd.read_csv("datasets/KDD_dataset/other_half_KDDTrain+.csv")
test_dataset = pd.read_csv("datasets/KDD_dataset/KDDTest+.csv")

In [17]:
train_features = ["duration", "protocol_type", "src_bytes", "dst_bytes", "count", "srv_count", "is_guest_login", "root_shell", "num_failed_logins", "" "class"]
protocol_map = {'tcp': 1, 'udp': 2, 'icmp': 3}

def Preprocess_GAN(train):
    # map protocol_type to number
    train["protocol_type"]=train["protocol_type"].map(protocol_map)
    # delete the columns that are not in the train_features
    trash = list(set(train.columns) - set(train_features))
    # print ("train.column: ", set(train.columns))
    # print ("train_features: ",set(train_features))
    # print ("trash", trash)
    
    #delete trash columns
    for t in trash:
        del train[t]
        
    # min max standardization
    numeric_columns = list(train.select_dtypes(include=['int', "float"]).columns) # select all columns that are numeric
    for c in numeric_columns:
        max_ = train[c].max()
        min_ = train[c].min()
        train[c] = train[c].map(lambda x: (x - min_) / (max_ - min_))


    #  1: annomaly; 0: normaly
    train["class"] = train["class"].map(lambda x: 1 if x == "anomaly" else 0)
    # get all rows of malicious traffic, and all columns except the last one
    raw_attack = np.array(train[train["class"] == 1])[:, :-1]
    # get all rows of benign traffic, and all columns except the last one
    normal = np.array(train[train["class"] == 0])[:, :-1]
    
    # get the true label of the train set
    true_label = train["class"]

    del train["class"]

    return train, raw_attack, normal, true_label

In [18]:
train_data,raw_attack,normal,true_label = Preprocess_GAN(train_dataset)


In [19]:
def CreateBatch_GAN(x, batch_size):
    # print("x.shape: ", x.shape) 
    # Comment - a là danh sách các số từ 0 -> len(x)
    a = list(range(len(x)))
   # print("a: ", a)
    # Comment - Xáo trộn a lên, đảo lộn vị trí các phần từ của a
    np.random.shuffle(a)
    # Comment - Xáo trộn các phần tử trong x
    x = x[a]
   # print("x: ", x)
    # Comment - Mảng các batch, mỗi batch có số phần tử là batch size
    batch_x = [x[batch_size * i: (i + 1) * batch_size, :] for i in range(len(x) // batch_size)]
    return batch_x

In [20]:
class Blackbox_IDS(nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.layer = nn.Sequential(
            #nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, input_dim*2),
            nn.Dropout(0.6),   
            #nn.ELU(),
            nn.LeakyReLU(True),
           # nn.BatchNorm1d(input_dim*2),
            nn.Linear(input_dim *2, input_dim *2),
            nn.Dropout(0.5),       
           # nn.ELU(),
#            nn.ReLU(True),
            nn.LeakyReLU(True),   
          # nn.BatchNorm1d(input_dim*2),
            nn.Linear(input_dim *2, input_dim//2),
            nn.Dropout(0.5),       
#            nn.ReLU(True),
        #  nn.ELU(),
            nn.LeakyReLU(True),
           #nn.BatchNorm1d(input_dim//2),
            nn.Linear(input_dim//2,input_dim//2),
            nn.Dropout(0.4),       
        #    nn.ELU(),
#            nn.ReLU(True),
            nn.LeakyReLU(True),            
            nn.Linear(input_dim//2,output_dim),
        )
        #nn.init.kaiming_normal_(self.layer.weight)
        self.output = nn.Sigmoid()
        #self.output = nn.Softmax()
    def forward(self,x):
        x = self.layer(x)
        return x

class Generator(nn.Module):
    def __init__(self,input_dim, output_dim):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_dim, input_dim//2),
            nn.ReLU(True),
            nn.Linear(input_dim // 2, input_dim//2),
            nn.ReLU(True),
            nn.Linear(input_dim // 2, input_dim//2),
            nn.ReLU(True),
            nn.Linear(input_dim // 2,input_dim//2),
            nn.ReLU(True),
            nn.Linear(input_dim//2,output_dim),
        )
    def forward(self,x):
        x = self.layer(x)
        return th.clamp(x,0.,1.)

class Discriminator(nn.Module):
    def __init__(self,input_dim, output_dim):
        super(Discriminator, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(input_dim, input_dim*2),
            nn.LeakyReLU(True),
            nn.Linear(input_dim * 2, input_dim),
            nn.LeakyReLU(True),
            #nn.Linear(input_dim*2 , input_dim*2),
            #nn.LeakyReLU(True),
            nn.Linear(input_dim,input_dim//2),
            nn.LeakyReLU(True),
            nn.Linear(input_dim//2,output_dim),
        )

    def forward(self,x):
        return self.layer(x)

In [21]:
def compute_gradient_penalty(D, normal_t, attack_t):
    alpha = th.Tensor(np.random.random((normal_t.shape[0], 1)))
    between_n_a = (alpha * normal_t + ((1 - alpha) * attack_t)).requires_grad_(True)
    d_between_n_a = D(between_n_a)
    adv = V(th.Tensor(normal_t.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = autograd.grad(
        outputs=d_between_n_a,
        inputs=between_n_a,
        grad_outputs=adv,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

train_dataset = pd.read_csv("datasets/KDD_dataset/other_half_KDDTrain+.csv")
test_dataset = pd.read_csv("datasets/KDD_dataset/KDDTest+.csv")

train_data,raw_attack,normal,true_label = Preprocess_GAN(train_dataset)

#DEFINE
BATCH_SIZE = 64 # Batch size
CRITIC_ITERS = 5 # For WGAN and WGAN-GP, number of critic iters per gen iter
LAMBDA = 10     # Gradient penalty lambda hyperparameter
MAX_EPOCH = 100 # How many generator iterations to train for
D_G_INPUT_DIM = len(train_data.columns) # 9 features
G_OUTPUT_DIM = len(train_data.columns) # 9 features
D_OUTPUT_DIM = 1
CLAMP = 0.01
LEARNING_RATE=0.0001

# print D_G_INPUT_DIM
# print("D_G_input DIM: ", D_G_INPUT_DIM)

# Load BlackBox IDS model 
ids_model = Blackbox_IDS(D_G_INPUT_DIM,2)
param = th.load('datasets/KDD_dataset/IDS.pth')
ids_model.load_state_dict(param)


generator = Generator(D_G_INPUT_DIM,G_OUTPUT_DIM)
print(100*'=')
print(generator)

discriminator = Discriminator(D_G_INPUT_DIM,D_OUTPUT_DIM)
print(100*'=')
print(discriminator)


#Optimization. Similar to Gradient Descent. https://viblo.asia/p/thuat-toan-toi-uu-adam-aWj53k8Q56m
optimizer_G = optim.RMSprop(generator.parameters(), LEARNING_RATE)
optimizer_D = optim.RMSprop(discriminator.parameters(), LEARNING_RATE)

# 由於不可能放入整個資料集，因此資料集會分批輸出（更小、相等的部分）。
batch_attack = CreateBatch_GAN(raw_attack,BATCH_SIZE)
d_losses,g_losses = [],[] #loss status
ids_model.eval()

generator.train()
discriminator.train()

cnt = -5
print("IDSGAN start training")
print("-"*100)
for epoch in range(MAX_EPOCH):
    # Comment - Mỗi train epoch tạo batch 1 lần
    normal_batch = CreateBatch_GAN(normal,BATCH_SIZE)
    epoch_g_loss = 0.
    epoch_d_loss = 0.
    c=0
    for nb in normal_batch:
        normal_b = th.Tensor(nb)
        #  Train Generator
        for p in discriminator.parameters():
            p.requires_grad = False

        optimizer_G.zero_grad()

        # random_traffic - Lay tu raw_attack ngau nhien n=BATCH_SIZE phan tu
        random_attack_traffic = raw_attack[np.random.randint(0,len(raw_attack),BATCH_SIZE)]
        # random_traffic_noised - Lay random_traffic + noise la gia tri random tu 0 - 1

        ###!! random_traffic_noised - Nhu vay random_traffic_noised co the la gia tri > 1
        # random_traffic_noised = random_attack_traffic + np.random.uniform(0,1,(BATCH_SIZE,D_G_INPUT_DIM))
        fixed_value = 0.5
        random_traffic_noised = random_attack_traffic + np.full((BATCH_SIZE, D_G_INPUT_DIM), fixed_value)
        z = V(th.Tensor(random_traffic_noised))
        adversarial_traffic = generator(z)
        
        D_pred= discriminator(adversarial_traffic) #điểm của generated output
        g_loss = -th.mean(D_pred)
        g_loss.backward()
        optimizer_G.step()

        epoch_g_loss += g_loss.item()
        # Train Discriminator
        for p in discriminator.parameters():
            p.requires_grad = True

        for c in range(CRITIC_ITERS):
            optimizer_D.zero_grad()
            for p in discriminator.parameters():
                p.data.clamp_(-CLAMP, CLAMP)

            # temp_data = raw_attack[np.random.randint(0,len(raw_attack),BATCH_SIZE)] + np.random.uniform(0,1,(BATCH_SIZE,D_G_INPUT_DIM))
            temp_data = raw_attack[np.random.randint(0,len(raw_attack),BATCH_SIZE)] + np.full((BATCH_SIZE, D_G_INPUT_DIM), fixed_value)
            z = V(th.Tensor(temp_data))
            adversarial_traffic = generator(z).detach()
            ids_input = th.cat((adversarial_traffic,normal_b))

            l = list(range(len(ids_input)))
            np.random.shuffle(l)
            ids_input = V(th.Tensor(ids_input[l]))
            ids_pred = ids_model(ids_input)
            ids_pred_label = th.argmax(nn.Sigmoid()(ids_pred),dim = 1).detach().numpy()

            pred_normal = ids_input.numpy()[ids_pred_label==0]
            pred_attack = ids_input.numpy()[ids_pred_label==1]

            if len(pred_attack) == 0:
                cnt += 1
                break

            D_normal = discriminator(V(th.Tensor(pred_normal)))
            D_attack= discriminator(V(th.Tensor(pred_attack)))

            loss_normal = th.mean(D_normal)
            loss_attack = th.mean(D_attack)
            #gradient_penalty = compute_gradient_penalty(discriminator, normal_b.data, adversarial_traffic.data)
            d_loss =  loss_attack - loss_normal #+ LAMBDA * gradient_penalty
            d_loss.backward()
            optimizer_D.step()
            epoch_d_loss += d_loss.item()

    d_losses.append(epoch_d_loss/CRITIC_ITERS)
    g_losses.append(epoch_g_loss)
    print(f"{epoch} : {epoch_g_loss} \t {epoch_d_loss/CRITIC_ITERS}")
'''
    if cnt >= 100:
        print("Not exist predicted attack traffic")
        break
'''

print("IDSGAN finish training")

th.save(generator.state_dict(), 'GAN_materials/testGAN/generator.pth')
th.save(discriminator.state_dict(), 'GAN_materials/testGAN/discriminator.pth')

plt.plot(d_losses,label = "D_loss")
plt.plot(g_losses, label = "G_loss")
plt.legend()
plt.show()

Generator(
  (layer): Sequential(
    (0): Linear(in_features=9, out_features=4, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=4, out_features=4, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=4, out_features=9, bias=True)
  )
)
Discriminator(
  (layer): Sequential(
    (0): Linear(in_features=9, out_features=18, bias=True)
    (1): LeakyReLU(negative_slope=True)
    (2): Linear(in_features=18, out_features=9, bias=True)
    (3): LeakyReLU(negative_slope=True)
    (4): Linear(in_features=9, out_features=4, bias=True)
    (5): LeakyReLU(negative_slope=True)
    (6): Linear(in_features=4, out_features=1, bias=True)
  )
)
IDSGAN start training
----------------------------------------------------------------------------------------------------
0 : 3.61980137534556 	 -0.00444200563

KeyboardInterrupt: 